In [6]:
import jenkins

# Login


In [14]:
url = ""
uname = "jenkins-admin"
passwd = ""
server = jenkins.Jenkins(url, username=uname, password=passwd)

In [ ]:
try:
    print(server.get_whoami())
except jenkins.JenkinsException as e:
    # print(e)
    print("HTTP ERROR 401 Unauthorized" in e.args[0])

In [3]:
server.get_whoami().get("id")

'jenkins-admin'

In [7]:
server.get_version()

'2.476'

# Jobs


## create


In [ ]:
server.create_job("empty", jenkins.EMPTY_CONFIG_XML)

In [ ]:
# 有一个缺点，不能跨文件夹拷贝
server.copy_job("empty", "empty_copy")

In [16]:
# 跨文件夹拷贝
source_job_config_xml = server.get_job_config("empty")
server.create_job("Template/empty_copy", source_job_config_xml)

In [ ]:
# 获取视图下的任务
server.get_jobs(view_name="ss")

## config


In [ ]:
# 重置Job的config
server.reconfig_job("empty_copy", jenkins.RECONFIG_XML)

In [ ]:
from jenkins import NotFoundException

try:
    job = server.get_job_config("empty")
    print(job)
except NotFoundException:
    print("未找到job")

In [ ]:
print(server.get_job_config("channel/channel/server"))

## get


In [9]:
server.assert_job_exists("empty")

In [130]:
print(server.job_exists("empty1"))
print(server.job_exists("empty"))

None
True


In [113]:
server.get_job_name("empty")

In [10]:
server.jobs_count()

18

In [ ]:
server.get_all_jobs()

In [ ]:
len(server.get_all_jobs())

In [ ]:
# 重要 包含了最近几次的build信息
server.get_job_info("empty")

In [6]:
# get by view
server.get_jobs(view_name="ss")

[]

## update


In [26]:
server.disable_job("empty")

In [27]:
server.enable_job("empty")

In [12]:
import xmltodict

def get_job_conf(job_name: str) -> dict:
    res = server.get_job_config(job_name)
    xml_parse = xmltodict.parse(res)
    return xml_parse


def reconfig(job_name, job_config: dict):
    config_xml = xmltodict.unparse(job_config)
    r = server.reconfig_job("empty", config_xml)
    print(r)


# 使用模板的配置 覆盖原本的配置
Template_job = get_job_conf("Template/channel-service")
reconfig("empty", Template_job)

None


## build


### execute build


In [ ]:
server.build_job("empty")
job = server.get_job_info("empty")

In [ ]:
# 带参构建
server.build_job("empty", {"param1": "test value 1"})

### build info


In [101]:
# 判断一个job是否在build
print(server.get_job_info("empty").get("inQueue"))  #  False
server.build_job("empty")
print(server.get_job_info("empty").get("inQueue"))  #  True

False
True
True


In [46]:
# 一个job最后一次构建id
last_build_number = server.get_job_info("empty")["lastCompletedBuild"]["number"]
last_build_number

2

In [ ]:
server.get_build_info("empty", 2)

### build number


In [96]:
server.get_job_info("empty")["nextBuildNumber"]

13

### build env


In [123]:
server.get_build_env_vars("server", number=33)

### build console output


In [149]:
server.get_build_console_output("empty", 13)

'Started by user jenkins-admin\nRunning as SYSTEM\nBuilding remotely on node-191 (BJPBS-PROD) in workspace /opt/jenkins/workspace/empty\nFinished: SUCCESS\n'

## delete


In [ ]:
server.delete_job("empty")
server.delete_job("empty_copy")

# Build Queue


In [90]:
server.build_job("empty")

319

In [ ]:
# 具有时效性,等构建任务完成后,很快会无法根据number获取任务
server.get_queue_item(number=317)

In [ ]:
queue = server.get_queue_info()
queue[0].get("task").get("name")

In [51]:
id = queue_info[0].get("id")
id

311

In [52]:
server.cancel_queue(id)

# Views


In [9]:
server.create_view("TEST_VIEW", jenkins.EMPTY_VIEW_CONFIG_XML)

In [ ]:
server.get_view_config("TEST_VIEW")

In [15]:
[i.get("name") for i in server.get_views()]

['TEST_VIEW', 'all', 'ss']

In [16]:
server.delete_view("TEST_VIEW")

# Nodes


In [140]:
server.get_nodes()

[{'name': 'Built-In Node', 'offline': False},
 {'name': 'node-191', 'offline': False}]

In [ ]:
server.get_node_info("node-191")

# Plugins


In [33]:
len(server.get_plugins_info())

76